In [1]:
!pip install flask psutil scikit-learn pyngrok -q

In [4]:

import flask
from flask import Flask, jsonify, render_template_string
import psutil
import numpy as np
import json
import time
import threading
from datetime import datetime
from collections import deque
import pickle
import os
from pyngrok import ngrok


ngrok.set_auth_token("39jma9KIB5JVIQzL4QDfQ0omiv1_5q9BSkhi4gc3sdYvwYSCm")


try:
    from sklearn.ensemble import IsolationForest
    from sklearn.preprocessing import StandardScaler
    sklearn_available = True
except ImportError:
    sklearn_available = False
    print("Warning: scikit-learn not available. Using mock anomaly detection.")

app = Flask(__name__)


PORT = 5000
METRICS_HISTORY_SIZE = 100
UPDATE_INTERVAL = 2  # seconds

# Color palette
COLORS = {
    "purple": "#7b2cbf",
    "blue": "#3a86ff",
    "gold": "#ffd166",
    "dark_bg": "#0b0d1a",
    "card_bg": "#12142e",
    "alert_bg": "#1a0b2e",
    "recommendation_bg": "#0b1a2e"
}

# Store metrics history
metrics_history = {
    "cpu": deque(maxlen=METRICS_HISTORY_SIZE),
    "memory": deque(maxlen=METRICS_HISTORY_SIZE),
    "disk": deque(maxlen=METRICS_HISTORY_SIZE),
    "timestamps": deque(maxlen=METRICS_HISTORY_SIZE)
}

# Store anomalies and recommendations
anomalies = deque(maxlen=20)
recommendations = deque(maxlen=20)

# System status
system_status = {
    "overall": "healthy",
    "last_check": datetime.now().isoformat(),
    "anomaly_count": 0,
    "cpu_status": "normal",
    "memory_status": "normal",
    "disk_status": "normal"
}


# AI Model - Isolation Forest Implementation

class AnomalyDetector:
    def __init__(self):
        self.model = None
        self.scaler = StandardScaler() if sklearn_available else None
        self.is_trained = False
        self.init_model()

    def init_model(self):
        """Initialize and train the Isolation Forest model"""
        if not sklearn_available:
            print("Using mock anomaly detector (scikit-learn not available)")
            return

        try:
            # Create synthetic normal data for training
            np.random.seed(42)
            n_samples = 1000

            # Generate normal operating ranges:
            # CPU: 5-40%, Memory: 20-70%, Disk: 10-60%
            normal_cpu = np.random.normal(20, 10, n_samples).clip(5, 40)
            normal_memory = np.random.normal(45, 15, n_samples).clip(20, 70)
            normal_disk = np.random.normal(35, 12, n_samples).clip(10, 60)

            X_train = np.column_stack([normal_cpu, normal_memory, normal_disk])

            # Scale the data
            X_train_scaled = self.scaler.fit_transform(X_train)

            # Train Isolation Forest
            self.model = IsolationForest(
                n_estimators=100,
                contamination=0.05,  # Expect 5% anomalies
                random_state=42,
                n_jobs=-1
            )
            self.model.fit(X_train_scaled)
            self.is_trained = True
            print("AI Model trained successfully with Isolation Forest")

        except Exception as e:
            print(f"Error training model: {e}")
            self.is_trained = False

    def detect_anomaly(self, cpu, memory, disk):
        """Detect if current metrics are anomalous"""
        if not sklearn_available or not self.is_trained:
            # Mock detection for demo purposes
            threshold = 80
            if cpu > threshold or memory > threshold or disk > threshold:
                return -1, 0.8 if cpu > threshold else 0.6
            return 1, 0.1

        try:
            # Prepare input
            X = np.array([[cpu, memory, disk]])
            X_scaled = self.scaler.transform(X)

            # Predict
            prediction = self.model.predict(X_scaled)[0]  # 1 = normal, -1 = anomaly

            # Get anomaly score
            scores = self.model.score_samples(X_scaled)
            anomaly_score = 1 - (1 / (1 + np.exp(-scores[0])))  # Convert to 0-1 scale

            return prediction, float(anomaly_score)

        except Exception as e:
            print(f"Error in anomaly detection: {e}")
            return 1, 0.1

# Initialize detector
detector = AnomalyDetector()


# System Monitoring Functions

def get_system_metrics():
    """Collect current system metrics"""
    try:
        # CPU usage
        cpu_percent = psutil.cpu_percent(interval=0.1)

        # Memory usage
        memory = psutil.virtual_memory()
        memory_percent = memory.percent

        # Disk usage
        disk = psutil.disk_usage('/')
        disk_percent = disk.percent

        # Network (optional)
        net_io = psutil.net_io_counters()
        network_sent = net_io.bytes_sent / (1024**2)  # MB
        network_recv = net_io.bytes_recv / (1024**2)  # MB

        return {
            "cpu": cpu_percent,
            "memory": memory_percent,
            "disk": disk_percent,
            "network_sent": round(network_sent, 2),
            "network_recv": round(network_recv, 2),
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        print(f"Error getting metrics: {e}")
        return {
            "cpu": 0,
            "memory": 0,
            "disk": 0,
            "network_sent": 0,
            "network_recv": 0,
            "timestamp": datetime.now().isoformat()
        }

def analyze_metrics(metrics):
    """Analyze metrics and generate recommendations"""
    cpu = metrics["cpu"]
    memory = metrics["memory"]
    disk = metrics["disk"]

    # Detect anomaly
    prediction, anomaly_score = detector.detect_anomaly(cpu, memory, disk)

    # Update system status
    system_status["last_check"] = metrics["timestamp"]

    # Determine individual resource status
    status_map = {"normal": "normal", "warning": "warning", "critical": "critical"}

    def get_resource_status(value, warning_thresh=75, critical_thresh=90):
        if value >= critical_thresh:
            return "critical"
        elif value >= warning_thresh:
            return "warning"
        return "normal"

    system_status["cpu_status"] = get_resource_status(cpu)
    system_status["memory_status"] = get_resource_status(memory)
    system_status["disk_status"] = get_resource_status(disk)

    # Determine overall status
    if any(s == "critical" for s in [system_status["cpu_status"],
                                      system_status["memory_status"],
                                      system_status["disk_status"]]):
        system_status["overall"] = "critical"
    elif any(s == "warning" for s in [system_status["cpu_status"],
                                      system_status["memory_status"],
                                      system_status["disk_status"]]):
        system_status["overall"] = "warning"
    else:
        system_status["overall"] = "healthy"

    # Generate anomaly alert if detected
    if prediction == -1 or anomaly_score > 0.7:
        system_status["anomaly_count"] += 1

        # Determine affected resource
        affected_resource = "CPU"
        max_val = cpu
        if memory > max_val:
            affected_resource = "Memory"
            max_val = memory
        if disk > max_val:
            affected_resource = "Disk"
            max_val = disk

        # Severity level
        severity = "HIGH" if anomaly_score > 0.8 else "MEDIUM" if anomaly_score > 0.6 else "LOW"

        # Create anomaly record
        anomaly = {
            "id": len(anomalies) + 1,
            "timestamp": metrics["timestamp"],
            "resource": affected_resource,
            "severity": severity,
            "score": round(anomaly_score, 3),
            "metrics": {
                "cpu": cpu,
                "memory": memory,
                "disk": disk
            }
        }
        anomalies.appendleft(anomaly)

        # Generate recommendation
        recommendation = generate_recommendation(affected_resource, cpu, memory, disk, severity)
        recommendations.appendleft(recommendation)

        return anomaly, recommendation

    return None, None

def generate_recommendation(resource, cpu, memory, disk, severity):
    """Generate intelligent recommendation based on anomaly"""
    timestamp = datetime.now().isoformat()

    if resource == "CPU":
        if cpu > 90:
            action = "Immediate action required. Consider: 1. Identify and kill runaway processes, 2. Scale up CPU resources, 3. Implement load balancing"
            cause = "Extremely high CPU usage indicating potential infinite loop or resource leak"
        elif cpu > 75:
            action = "Monitor closely. Suggested actions: 1. Check top processes (ps aux --sort=-%cpu), 2. Optimize application code, 3. Consider horizontal scaling"
            cause = "High CPU usage may indicate inefficient code or increased load"
        else:
            action = "Continue monitoring. Recommendation: 1. Review CPU usage patterns, 2. Set up alerts for threshold breaches"
            cause = "Moderate CPU anomaly detected"

    elif resource == "Memory":
        if memory > 90:
            action = "Critical memory situation. Immediate actions: 1. Check for memory leaks, 2. Restart affected services, 3. Increase system memory, 4. Clear caches"
            cause = "Memory exhaustion may lead to system instability or crashes"
        elif memory > 75:
            action = "Memory pressure detected. Recommendations: 1. Analyze memory usage by process, 2. Optimize memory allocation, 3. Implement garbage collection tuning"
            cause = "High memory usage may indicate memory leak or insufficient RAM"
        else:
            action = "Investigate memory patterns. Suggestion: 1. Monitor memory growth over time, 2. Review application memory settings"
            cause = "Unusual memory usage pattern detected"

    else:  # Disk
        if disk > 90:
            action = "Disk almost full! Critical actions: 1. Clean up unnecessary files, 2. Expand disk storage, 3. Archive old logs, 4. Monitor for rapid disk fill"
            cause = "Disk space exhaustion may cause system failures and data loss"
        elif disk > 75:
            action = "Disk space warning. Recommendations: 1. Identify large files/directories, 2. Implement log rotation, 3. Consider offloading old data"
            cause = "High disk usage may impact system performance and stability"
        else:
            action = "Review disk usage patterns. Suggestion: 1. Set up automated cleanup scripts, 2. Monitor disk growth rate"
            cause = "Unusual disk usage pattern detected"

    return {
        "id": len(recommendations) + 1,
        "timestamp": timestamp,
        "resource": resource,
        "severity": severity,
        "action": action,
        "cause": cause,
        "metrics": {
            "cpu": cpu,
            "memory": memory,
            "disk": disk
        }
    }


# Background Monitoring Thread

def background_monitoring():
    """Continuously monitor system in background"""
    while True:
        try:
            # Get current metrics
            current_metrics = get_system_metrics()

            # Update history
            metrics_history["cpu"].append(current_metrics["cpu"])
            metrics_history["memory"].append(current_metrics["memory"])
            metrics_history["disk"].append(current_metrics["disk"])
            metrics_history["timestamps"].append(
                datetime.now().strftime("%H:%M:%S")
            )

            # Analyze for anomalies
            analyze_metrics(current_metrics)

            time.sleep(UPDATE_INTERVAL)

        except Exception as e:
            print(f"Error in background monitoring: {e}")
            time.sleep(UPDATE_INTERVAL)

# Start monitoring thread
monitor_thread = threading.Thread(target=background_monitoring, daemon=True)
monitor_thread.start()


# Flask Routes

@app.route('/')
def index():
    """Serve the main dashboard"""
    return render_template_string(HTML_TEMPLATE, colors=COLORS)

@app.route('/metrics')
def get_metrics():
    """API endpoint for current metrics"""
    current_metrics = get_system_metrics()

    # Get recent history for charts
    history = {
        "cpu": list(metrics_history["cpu"]),
        "memory": list(metrics_history["memory"]),
        "disk": list(metrics_history["disk"]),
        "timestamps": list(metrics_history["timestamps"])
    }

    # Get latest anomaly and recommendation
    latest_anomaly = anomalies[0] if anomalies else None
    latest_recommendation = recommendations[0] if recommendations else None

    return jsonify({
        "current": current_metrics,
        "history": history,
        "status": system_status,
        "latest_anomaly": latest_anomaly,
        "latest_recommendation": latest_recommendation,
        "anomaly_count": len(anomalies),
        "colors": COLORS
    })

@app.route('/anomalies')
def get_anomalies():
    """Get all detected anomalies"""
    return jsonify({
        "anomalies": list(anomalies),
        "count": len(anomalies)
    })

@app.route('/recommendations')
def get_recommendations():
    """Get all recommendations"""
    return jsonify({
        "recommendations": list(recommendations),
        "count": len(recommendations)
    })

@app.route('/health')
def health_check():
    """Health check endpoint"""
    return jsonify({
        "status": "healthy",
        "service": "AI DevOps Monitoring Agent",
        "timestamp": datetime.now().isoformat(),
        "model_status": "trained" if detector.is_trained else "training_required"
    })

# ============================================
# HTML Template for Dashboard
# ============================================
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>AI DevOps Monitoring Agent</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/js/all.min.js"></script>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }

        :root {
            --purple: {{ colors.purple }};
            --blue: {{ colors.blue }};
            --gold: {{ colors.gold }};
            --dark-bg: {{ colors.dark_bg }};
            --card-bg: {{ colors.card_bg }};
            --alert-bg: {{ colors.alert_bg }};
            --recommendation-bg: {{ colors.recommendation_bg }};
        }

        body {
            background-color: var(--dark-bg);
            color: #ffffff;
            min-height: 100vh;
            overflow-x: hidden;
        }

        .container {
            max-width: 1400px;
            margin: 0 auto;
            padding: 20px;
        }

        /* Header */
        .header {
            text-align: center;
            padding: 30px 0;
            margin-bottom: 30px;
            background: linear-gradient(135deg, var(--purple) 0%, var(--blue) 100%);
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(123, 44, 191, 0.3);
            position: relative;
            overflow: hidden;
        }

        .header::before {
            content: '';
            position: absolute;
            top: -50%;
            left: -50%;
            width: 200%;
            height: 200%;
            background: radial-gradient(circle, rgba(255, 255, 255, 0.1) 1px, transparent 1px);
            background-size: 30px 30px;
            animation: float 20s linear infinite;
            z-index: 0;
        }

        @keyframes float {
            0% { transform: translate(0, 0) rotate(0deg); }
            100% { transform: translate(-30px, -30px) rotate(360deg); }
        }

        .header h1 {
            font-size: 2.8rem;
            font-weight: 800;
            margin-bottom: 10px;
            position: relative;
            z-index: 1;
            text-shadow: 0 2px 10px rgba(0, 0, 0, 0.3);
        }

        .header p {
            font-size: 1.2rem;
            opacity: 0.9;
            max-width: 800px;
            margin: 0 auto;
            position: relative;
            z-index: 1;
        }

        /* Status Badge */
        .status-badge {
            display: inline-block;
            padding: 8px 20px;
            border-radius: 50px;
            font-weight: bold;
            margin-top: 15px;
            background: rgba(0, 0, 0, 0.3);
            border: 2px solid var(--gold);
            position: relative;
            z-index: 1;
        }

        /* Metrics Cards */
        .metrics-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
            gap: 25px;
            margin-bottom: 30px;
        }

        .metric-card {
            background: var(--card-bg);
            border-radius: 15px;
            padding: 25px;
            box-shadow: 0 8px 25px rgba(0, 0, 0, 0.2);
            border: 2px solid;
            border-image: linear-gradient(45deg, var(--gold), var(--purple)) 1;
            transition: transform 0.3s, box-shadow 0.3s;
            position: relative;
            overflow: hidden;
            height: 180px;
            display: flex;
            flex-direction: column;
            justify-content: center;
        }

        .metric-card:hover {
            transform: translateY(-5px);
            box-shadow: 0 15px 35px rgba(123, 44, 191, 0.4);
        }

        .metric-card::before {
            content: '';
            position: absolute;
            top: 0;
            left: 0;
            right: 0;
            height: 5px;
            background: linear-gradient(90deg, var(--purple), var(--blue));
        }

        .metric-header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 15px;
        }

        .metric-title {
            font-size: 1.1rem;
            color: #aaa;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 1px;
        }

        .metric-icon {
            font-size: 1.8rem;
            color: var(--gold);
        }

        .metric-value {
            font-size: 2.8rem;
            font-weight: 800;
            margin-bottom: 10px;
            color: var(--blue);
        }

        .metric-status {
            display: inline-block;
            padding: 5px 15px;
            border-radius: 20px;
            font-size: 0.9rem;
            font-weight: 600;
            width: fit-content;
        }

        .status-normal { background: rgba(0, 255, 0, 0.1); color: #00ff00; border: 1px solid #00ff00; }
        .status-warning { background: rgba(255, 165, 0, 0.1); color: #ffa500; border: 1px solid #ffa500; }
        .status-critical { background: rgba(255, 0, 0, 0.1); color: #ff4444; border: 1px solid #ff4444; }

        /* Charts - Horizontal Layout */
        .charts-row {
            display: flex;
            gap: 25px;
            margin-bottom: 30px;
        }

        .chart-card {
            background: var(--card-bg);
            border-radius: 15px;
            padding: 25px;
            box-shadow: 0 8px 25px rgba(0, 0, 0, 0.2);
            border: 1px solid rgba(255, 209, 102, 0.1);
            flex: 1;
            min-height: 350px;
            display: flex;
            flex-direction: column;
        }

        .chart-card.horizontal {
            flex: 1;
            min-width: 0; /* Prevents flex items from overflowing */
        }

        .chart-title {
            font-size: 1.3rem;
            margin-bottom: 20px;
            color: var(--gold);
            display: flex;
            align-items: center;
            gap: 10px;
            flex-shrink: 0;
        }

        .chart-title i {
            color: var(--purple);
        }

        .chart-container {
            flex: 1;
            min-height: 0;
            position: relative;
        }

        /* Alerts & Recommendations */
        .alerts-container {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 25px;
            margin-bottom: 40px;
        }

        .alerts-panel, .recommendations-panel {
            border-radius: 15px;
            padding: 25px;
            box-shadow: 0 8px 25px rgba(0, 0, 0, 0.3);
            min-height: 350px;
        }

        .alerts-panel {
            background: var(--alert-bg);
            border: 1px solid rgba(123, 44, 191, 0.3);
        }

        .recommendations-panel {
            background: var(--recommendation-bg);
            border: 1px solid rgba(58, 134, 255, 0.3);
        }

        .panel-title {
            font-size: 1.5rem;
            margin-bottom: 20px;
            color: var(--gold);
            display: flex;
            align-items: center;
            gap: 10px;
        }

        .alerts-content, .recommendations-content {
            max-height: 280px;
            overflow-y: auto;
            padding-right: 10px;
        }

        .alert-item, .recommendation-item {
            background: rgba(0, 0, 0, 0.3);
            border-radius: 10px;
            padding: 20px;
            margin-bottom: 15px;
            border-left: 5px solid;
        }

        .alert-item {
            border-left-color: #ff4444;
        }

        .recommendation-item {
            border-left-color: var(--blue);
        }

        .alert-header, .recommendation-header {
            display: flex;
            justify-content: space-between;
            margin-bottom: 10px;
        }

        .alert-resource, .recommendation-resource {
            font-weight: 700;
            font-size: 1.1rem;
        }

        .alert-severity, .recommendation-severity {
            padding: 3px 10px;
            border-radius: 15px;
            font-size: 0.8rem;
            font-weight: 600;
        }

        .severity-high { background: rgba(255, 0, 0, 0.2); color: #ff7777; }
        .severity-medium { background: rgba(255, 165, 0, 0.2); color: #ffb347; }
        .severity-low { background: rgba(255, 255, 0, 0.2); color: #ffff77; }

        .alert-metrics, .recommendation-metrics {
            display: flex;
            gap: 15px;
            margin-top: 10px;
            font-size: 0.9rem;
            color: #aaa;
            flex-wrap: wrap;
        }

        .metric-tag {
            background: rgba(255, 255, 255, 0.1);
            padding: 3px 10px;
            border-radius: 12px;
        }

        .no-data {
            text-align: center;
            padding: 40px;
            color: #777;
            font-style: italic;
        }

        /* Footer */
        .footer {
            text-align: center;
            padding: 20px;
            margin-top: 40px;
            border-top: 1px solid rgba(255, 255, 255, 0.1);
            color: #777;
            font-size: 0.9rem;
        }

        .update-time {
            color: var(--gold);
            font-weight: 600;
        }

        /* Responsive */
        @media (max-width: 1200px) {
            .charts-row {
                flex-direction: column;
            }

            .chart-card.horizontal {
                min-height: 300px;
            }
        }

        @media (max-width: 900px) {
            .alerts-container {
                grid-template-columns: 1fr;
            }

            .metrics-grid {
                grid-template-columns: repeat(2, 1fr);
            }
        }

        @media (max-width: 768px) {
            .header h1 {
                font-size: 2rem;
            }

            .metric-value {
                font-size: 2.2rem;
            }

            .metrics-grid {
                grid-template-columns: 1fr;
            }

            .chart-card {
                min-height: 300px;
            }
        }

        /* Animations */
        @keyframes pulse {
            0% { box-shadow: 0 0 0 0 rgba(255, 209, 102, 0.7); }
            70% { box-shadow: 0 0 0 10px rgba(255, 209, 102, 0); }
            100% { box-shadow: 0 0 0 0 rgba(255, 209, 102, 0); }
        }

        .pulse {
            animation: pulse 2s infinite;
        }

        /* Scrollbar */
        ::-webkit-scrollbar {
            width: 8px;
        }

        ::-webkit-scrollbar-track {
            background: rgba(0, 0, 0, 0.2);
        }

        ::-webkit-scrollbar-thumb {
            background: linear-gradient(var(--purple), var(--blue));
            border-radius: 4px;
        }

        ::-webkit-scrollbar-thumb:hover {
            background: linear-gradient(var(--blue), var(--purple));
        }
    </style>
</head>
<body>
    <div class="container">
        <!-- Header -->
        <div class="header">
            <h1><i class="fas fa-robot"></i>DevOps Monitoring Agent</h1>
            <p>Real-Time Anomaly Detection & Intelligent Recommendations</p>
            <div class="status-badge">
                System Status: <span id="overall-status">LOADING...</span>
            </div>
        </div>

        <!-- Metrics Cards -->
        <div class="metrics-grid">
            <div class="metric-card">
                <div class="metric-header">
                    <div class="metric-title">CPU USAGE</div>
                    <div class="metric-icon"><i class="fas fa-microchip"></i></div>
                </div>
                <div class="metric-value" id="cpu-value">0.0%</div>
                <div class="metric-status status-normal" id="cpu-status">NORMAL</div>
            </div>

            <div class="metric-card">
                <div class="metric-header">
                    <div class="metric-title">MEMORY USAGE</div>
                    <div class="metric-icon"><i class="fas fa-memory"></i></div>
                </div>
                <div class="metric-value" id="memory-value">0.0%</div>
                <div class="metric-status status-normal" id="memory-status">NORMAL</div>
            </div>

            <div class="metric-card">
                <div class="metric-header">
                    <div class="metric-title">DISK USAGE</div>
                    <div class="metric-icon"><i class="fas fa-hdd"></i></div>
                </div>
                <div class="metric-value" id="disk-value">0.0%</div>
                <div class="metric-status status-normal" id="disk-status">NORMAL</div>
            </div>

            <div class="metric-card">
                <div class="metric-header">
                    <div class="metric-title">SYSTEM HEALTH</div>
                    <div class="metric-icon"><i class="fas fa-heartbeat"></i></div>
                </div>
                <div class="metric-value" id="anomaly-count">0</div>
                <div class="metric-status status-normal" id="system-health">HEALTHY</div>
            </div>
        </div>

        <!-- Charts - Horizontal Row 1 -->
        <div class="charts-row">
            <div class="chart-card horizontal">
                <div class="chart-title"><i class="fas fa-chart-line"></i> CPU Usage Over Time</div>
                <div class="chart-container">
                    <canvas id="cpu-chart"></canvas>
                </div>
            </div>

            <div class="chart-card horizontal">
                <div class="chart-title"><i class="fas fa-chart-area"></i> Memory Usage Over Time</div>
                <div class="chart-container">
                    <canvas id="memory-chart"></canvas>
                </div>
            </div>
        </div>

        <!-- Charts - Horizontal Row 2 -->
        <div class="charts-row">
            <div class="chart-card horizontal">
                <div class="chart-title"><i class="fas fa-hdd"></i> Disk Usage Over Time</div>
                <div class="chart-container">
                    <canvas id="disk-chart"></canvas>
                </div>
            </div>

            <div class="chart-card horizontal">
                <div class="chart-title"><i class="fas fa-network-wired"></i> Network Activity</div>
                <div class="chart-container">
                    <canvas id="network-chart"></canvas>
                </div>
            </div>
        </div>

        <!-- Alerts & Recommendations -->
        <div class="alerts-container">
            <!-- Alerts Panel -->
            <div class="alerts-panel">
                <div class="panel-title"><i class="fas fa-exclamation-triangle"></i> AI Anomaly Alerts</div>
                <div class="alerts-content" id="alerts-list">
                    <div class="no-data">No anomalies detected. System is running normally.</div>
                </div>
            </div>

            <!-- Recommendations Panel -->
            <div class="recommendations-panel">
                <div class="panel-title"><i class="fas fa-lightbulb"></i> AI Recommendations</div>
                <div class="recommendations-content" id="recommendations-list">
                    <div class="no-data">No recommendations available. Continue monitoring.</div>
                </div>
            </div>
        </div>

        <!-- Footer -->
        <div class="footer">
            <p>AI DevOps Monitoring Agent • Real-Time Anomaly Detection using Isolation Forest</p>
            <p>Last Updated: <span class="update-time" id="update-time">--:--:--</span></p>
            <p>Anomalies Detected: <span id="total-anomalies">0</span> • Model: Isolation Forest (Unsupervised)</p>
        </div>
    </div>

    <script>
        // Initialize charts
        const cpuCtx = document.getElementById('cpu-chart').getContext('2d');
        const memoryCtx = document.getElementById('memory-chart').getContext('2d');
        const diskCtx = document.getElementById('disk-chart').getContext('2d');
        const networkCtx = document.getElementById('network-chart').getContext('2d');

        const chartOptions = {
            responsive: true,
            maintainAspectRatio: false,
            plugins: {
                legend: {
                    display: true,
                    labels: {
                        color: '#aaa',
                        font: {
                            size: 12
                        }
                    }
                },
                tooltip: {
                    backgroundColor: 'rgba(0, 0, 0, 0.8)',
                    titleColor: '#fff',
                    bodyColor: '#fff',
                    borderColor: '#7b2cbf',
                    borderWidth: 1
                }
            },
            scales: {
                y: {
                    beginAtZero: true,
                    max: 100,
                    grid: {
                        color: 'rgba(255, 255, 255, 0.1)',
                        drawBorder: false
                    },
                    ticks: {
                        color: '#aaa',
                        font: {
                            size: 11
                        }
                    }
                },
                x: {
                    grid: {
                        color: 'rgba(255, 255, 255, 0.05)',
                        drawBorder: false
                    },
                    ticks: {
                        color: '#aaa',
                        maxTicksLimit: 8,
                        font: {
                            size: 11
                        }
                    }
                }
            },
            elements: {
                line: {
                    tension: 0.4
                },
                point: {
                    radius: 3,
                    hoverRadius: 6
                }
            }
        };

        const cpuChart = new Chart(cpuCtx, {
            type: 'line',
            data: {
                labels: [],
                datasets: [{
                    label: 'CPU Usage %',
                    data: [],
                    borderColor: '#3a86ff',
                    backgroundColor: 'rgba(58, 134, 255, 0.1)',
                    borderWidth: 3,
                    fill: true,
                    pointBackgroundColor: '#7b2cbf',
                    pointBorderColor: '#ffffff',
                    pointBorderWidth: 2
                }]
            },
            options: chartOptions
        });

        const memoryChart = new Chart(memoryCtx, {
            type: 'line',
            data: {
                labels: [],
                datasets: [{
                    label: 'Memory Usage %',
                    data: [],
                    borderColor: '#ffd166',
                    backgroundColor: 'rgba(255, 209, 102, 0.1)',
                    borderWidth: 3,
                    fill: true,
                    pointBackgroundColor: '#7b2cbf',
                    pointBorderColor: '#ffffff',
                    pointBorderWidth: 2
                }]
            },
            options: chartOptions
        });

        const diskChart = new Chart(diskCtx, {
            type: 'line',
            data: {
                labels: [],
                datasets: [{
                    label: 'Disk Usage %',
                    data: [],
                    borderColor: '#7b2cbf',
                    backgroundColor: 'rgba(123, 44, 191, 0.1)',
                    borderWidth: 3,
                    fill: true,
                    pointBackgroundColor: '#ffd166',
                    pointBorderColor: '#ffffff',
                    pointBorderWidth: 2
                }]
            },
            options: chartOptions
        });

        const networkChart = new Chart(networkCtx, {
            type: 'line',
            data: {
                labels: [],
                datasets: [
                    {
                        label: 'Network Sent (MB)',
                        data: [],
                        borderColor: '#00ff9d',
                        backgroundColor: 'rgba(0, 255, 157, 0.1)',
                        borderWidth: 2,
                        fill: true
                    },
                    {
                        label: 'Network Received (MB)',
                        data: [],
                        borderColor: '#ff6b9d',
                        backgroundColor: 'rgba(255, 107, 157, 0.1)',
                        borderWidth: 2,
                        fill: true
                    }
                ]
            },
            options: {
                ...chartOptions,
                scales: {
                    ...chartOptions.scales,
                    y: {
                        ...chartOptions.scales.y,
                        max: null
                    }
                }
            }
        });

        // Update dashboard data
        async function updateDashboard() {
            try {
                const response = await fetch('/metrics');
                const data = await response.json();

                // Update metric cards
                document.getElementById('cpu-value').textContent = data.current.cpu.toFixed(1) + '%';
                document.getElementById('memory-value').textContent = data.current.memory.toFixed(1) + '%';
                document.getElementById('disk-value').textContent = data.current.disk.toFixed(1) + '%';
                document.getElementById('anomaly-count').textContent = data.anomaly_count;

                // Update status badges
                updateStatusBadge('cpu', data.status.cpu_status);
                updateStatusBadge('memory', data.status.memory_status);
                updateStatusBadge('disk', data.status.disk_status);
                updateSystemHealth(data.status.overall);

                // Update charts
                updateChart(cpuChart, data.history.timestamps, data.history.cpu);
                updateChart(memoryChart, data.history.timestamps, data.history.memory);
                updateChart(diskChart, data.history.timestamps, data.history.disk);

                // Update network chart with mock data for now
                updateNetworkChart(data.history.timestamps);

                // Update alerts and recommendations
                updateAlerts(data.latest_anomaly, data.anomalies);
                updateRecommendations(data.latest_recommendation, data.recommendations);

                // Update footer
                document.getElementById('update-time').textContent =
                    new Date(data.current.timestamp).toLocaleTimeString();
                document.getElementById('total-anomalies').textContent = data.anomaly_count;

                // Update overall status
                const overallStatus = document.getElementById('overall-status');
                overallStatus.textContent = data.status.overall.toUpperCase();
                overallStatus.style.color = getStatusColor(data.status.overall);

            } catch (error) {
                console.error('Error updating dashboard:', error);
            }
        }

        function updateStatusBadge(resource, status) {
            const element = document.getElementById(resource + '-status');
            element.textContent = status.toUpperCase();
            element.className = 'metric-status status-' + status.toLowerCase();

            // Add pulse animation for critical status
            if (status === 'critical') {
                element.classList.add('pulse');
            } else {
                element.classList.remove('pulse');
            }
        }

        function updateSystemHealth(status) {
            const element = document.getElementById('system-health');
            element.textContent = status.toUpperCase();
            element.className = 'metric-status status-' + status.toLowerCase();
        }

        function getStatusColor(status) {
            switch(status.toLowerCase()) {
                case 'healthy': return '#00ff00';
                case 'warning': return '#ffa500';
                case 'critical': return '#ff4444';
                default: return '#ffffff';
            }
        }

        function updateChart(chart, labels, data) {
            if (labels.length > 20) {
                chart.data.labels = labels.slice(-20);
                chart.data.datasets[0].data = data.slice(-20);
            } else {
                chart.data.labels = labels;
                chart.data.datasets[0].data = data;
            }
            chart.update('none');
        }

        function updateNetworkChart(labels) {
            // Generate mock network data for demonstration
            const sentData = [];
            const recvData = [];

            for (let i = 0; i < labels.length; i++) {
                sentData.push(Math.random() * 50 + 10);
                recvData.push(Math.random() * 30 + 5);
            }

            if (labels.length > 20) {
                networkChart.data.labels = labels.slice(-20);
                networkChart.data.datasets[0].data = sentData.slice(-20);
                networkChart.data.datasets[1].data = recvData.slice(-20);
            } else {
                networkChart.data.labels = labels;
                networkChart.data.datasets[0].data = sentData;
                networkChart.data.datasets[1].data = recvData;
            }
            networkChart.update('none');
        }

        function updateAlerts(latestAnomaly, allAnomalies) {
            const container = document.getElementById('alerts-list');

            if (!latestAnomaly) {
                container.innerHTML = '<div class="no-data">No anomalies detected. System is running normally.</div>';
                return;
            }

            let alertsHTML = '';
            const anomaliesToShow = allAnomalies && allAnomalies.length > 0 ?
                Array.from(allAnomalies).slice(0, 3) : [latestAnomaly];

            anomaliesToShow.forEach(alert => {
                const time = new Date(alert.timestamp).toLocaleTimeString([], {hour: '2-digit', minute:'2-digit'});
                alertsHTML += `
                    <div class="alert-item">
                        <div class="alert-header">
                            <div class="alert-resource">${alert.resource} Anomaly</div>
                            <div class="alert-severity severity-${alert.severity.toLowerCase()}">
                                ${alert.severity}
                            </div>
                        </div>
                        <div class="alert-message">Detected at ${time} | Score: ${alert.score}</div>
                        <div class="alert-metrics">
                            <div class="metric-tag">CPU: ${alert.metrics.cpu.toFixed(1)}%</div>
                            <div class="metric-tag">Memory: ${alert.metrics.memory.toFixed(1)}%</div>
                            <div class="metric-tag">Disk: ${alert.metrics.disk.toFixed(1)}%</div>
                        </div>
                    </div>
                `;
            });

            container.innerHTML = alertsHTML;
        }

        function updateRecommendations(latestRec, allRecs) {
            const container = document.getElementById('recommendations-list');

            if (!latestRec) {
                container.innerHTML = '<div class="no-data">No recommendations available. Continue monitoring.</div>';
                return;
            }

            let recsHTML = '';
            const recsToShow = allRecs && allRecs.length > 0 ?
                Array.from(allRecs).slice(0, 3) : [latestRec];

            recsToShow.forEach(rec => {
                const time = new Date(rec.timestamp).toLocaleTimeString([], {hour: '2-digit', minute:'2-digit'});
                recsHTML += `
                    <div class="recommendation-item">
                        <div class="recommendation-header">
                            <div class="recommendation-resource">${rec.resource} Recommendation</div>
                            <div class="recommendation-severity severity-${rec.severity.toLowerCase()}">
                                ${rec.severity}
                            </div>
                        </div>
                        <div class="recommendation-cause"><strong>Cause:</strong> ${rec.cause}</div>
                        <div class="recommendation-action"><strong>Action:</strong> ${rec.action}</div>
                        <div class="recommendation-metrics">
                            <div class="metric-tag">Time: ${time}</div>
                            <div class="metric-tag">CPU: ${rec.metrics.cpu.toFixed(1)}%</div>
                            <div class="metric-tag">Memory: ${rec.metrics.memory.toFixed(1)}%</div>
                        </div>
                    </div>
                `;
            });

            container.innerHTML = recsHTML;
        }

        // Initialize and update dashboard every 2 seconds
        document.addEventListener('DOMContentLoaded', () => {
            updateDashboard();
            setInterval(updateDashboard, 2000);
        });
    </script>
</body>
</html>
'''


# Main Execution

if __name__ == '__main__':
    print("=" * 60)
    print("AI DevOps Monitoring Agent")
    print("Real-Time Anomaly Detection & Intelligent Recommendations")
    print("=" * 60)

    print(f"\n Starting Flask server on port {PORT}...")

    # Start ngrok tunnel for public access
    try:
        ngrok_tunnel = ngrok.connect(PORT)
        public_url = ngrok_tunnel.public_url
        print(f"\n🌐 PUBLIC DASHBOARD URL: {public_url}")
        print("Dashboard is now publicly accessible!")
        print("   Share this URL with judges/team members")
    except Exception as e:
        print(f"\n⚠️  Ngrok tunnel failed: {e}")
        print(f"   Running locally only: http://127.0.0.1:{PORT}")
        public_url = f"http://127.0.0.1:{PORT}"

    print("\n Monitoring started...")
    print("• Collecting system metrics every 2 seconds")
    print("• AI Model: Isolation Forest (Anomaly Detection)")
    print("• Dashboard updating in real-time")
    print("• Anomaly detection & intelligent recommendations active")
    print("\n System Status:")
    print(f"   - Port: {PORT}")
    print(f"   - URL: {public_url}")
    print(f"   - AI Model: {'Trained' if detector.is_trained else 'Training Required'}")
    print("\n Press Ctrl+C to stop the monitoring agent\n")

    # Run Flask app
    app.run(host='0.0.0.0', port=PORT, debug=False)

AI Model trained successfully with Isolation Forest
AI DevOps Monitoring Agent
Real-Time Anomaly Detection & Intelligent Recommendations

 Starting Flask server on port 5000...

🌐 PUBLIC DASHBOARD URL: https://1bfe-35-197-148-247.ngrok-free.app
Dashboard is now publicly accessible!
   Share this URL with judges/team members

 Monitoring started...
• Collecting system metrics every 2 seconds
• AI Model: Isolation Forest (Anomaly Detection)
• Dashboard updating in real-time
• Anomaly detection & intelligent recommendations active

 System Status:
   - Port: 5000
   - URL: https://1bfe-35-197-148-247.ngrok-free.app
   - AI Model: Trained

 Press Ctrl+C to stop the monitoring agent

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:54:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:02] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:02] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:04] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:06] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:08] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:10] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:12] "GET /metrics HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Feb/2026 12:55:14] "GET /metrics HTTP/1.1" 